IMPLEMENTACION SKLEARN

In [8]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

RANDOM_STATE = 42
TRAIN_PERCENTAGE = 0.8
NUMBER_COMPONENTS = 20
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

TRANSFORMAMOS LOS DATOS A UNA MATRIZ DISPERSA

In [9]:
# Find the number of users and items
num_users = max(train["user"].max(), test["user"].max()) + 1
num_items = max(train["item"].max(), test["item"].max()) + 1

# Convert to CSR sparse matrices
train_sparse = csr_matrix((train["rating"], (train["user"], train["item"])), shape=(num_users, num_items))

Entrenamiento

In [10]:
import numpy as np
from sklearn.decomposition import NMF
model = NMF(n_components=NUMBER_COMPONENTS, init='random', random_state=42)
W = model.fit_transform(train_sparse)
H = model.components_

print("Reconstruction Error:", model.reconstruction_err_)


Reconstruction Error: 4708.831741838203


Realizamos las predicciones

In [17]:
def predict_value(user, item, W, H):
    """ Predict missing value for a given user-item pair """
    if user >= W.shape[0] or item >= H.shape[1]:
        print(f"Usuario con id {user} o item con id {item} no encontrado")
        return None  # User or item not in training set

    return np.dot(W[user, :], H[:, item])

# Example: Predict value for user 1 and item 0
predicted_value = predict_value(300, 10, W, H)
print(f"Predicted value for user 1 and item 0: {predicted_value:.4f}")

Predicted value for user 1 and item 0: 0.0000


Generamos el test

In [20]:
submition = pd.DataFrame(columns=['ID', 'rating'])
# Iterate row by row
for index, row in test.iterrows():
    predicted_value = predict_value(row['user'], row['item'], W, H)
    new_row = pd.DataFrame([{'ID': row['ID'], 'rating': predicted_value}])
    submition = pd.concat([submition, new_row], ignore_index=True)
    
submition.to_csv('./data/submission_NMF.csv', index=False)

C:\Users\pable\AppData\Local\Temp\ipykernel_12472\2275328389.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  submition = pd.concat([submition, new_row], ignore_index=True)
